## Uavs Collision

This notebook implements a uav logic instead of a plan and decisions are done online following that logic

In [1]:
import os
import subprocess
from pymavlink import mavutil
import numpy as np
# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns,global2local
from helpers.visualization.gazebo_world import update_world
from helpers.visualization.plots import plot_3d_interactive
from helpers.math import manhattan_distance
from helpers.navegation_logic import find_path,find_best_waypoint
from plans.plans import make_plan_from_wps

## Kill all related process

In [2]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

## Paths

In [3]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/attack_city.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

# Choose Simulator

In [4]:
simulator ='QGroundControl' #'gazebo' # 

# Create Plans

In [5]:
offsets = [(40, -10, 0, 0),(15, -10, 0, 0)]#
n_uavs = len(offsets)


homes=np.array([offset[:3] for offset in offsets])

global_paths=[np.array([[40, -10, 5],[15, -10, 5]]),np.array([[15, -10, 5],[40, -10, 5]])]
local_paths=[global2local(path, home,pairwise=True) for path,home in zip(global_paths,homes)]
plans=[make_plan_from_wps(wps=path,alt=5,wp_margin=0.5,navegation_speed=10) for path in local_paths]
markers = {f'waypoints': {'pos':global_paths[0],'color':'Green'}}


plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)



if simulator == 'QGroundControl':
    home_position=(-35.3633245,149.1652241,0,0)
    spawns=find_spawns(home_position, offsets)

Plan 'Waypoints Trajectory' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Navigation Speed' created — no connection yet 🧩
Step 'Set speed to 10.00 m/s' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'Take Off' created — no connection yet 🧩
Step 'take off' created — no connection yet 🧩
Action 'Go local' created — no connection yet 🧩
Step 'go to -> (0, 0, 5)' created — no connection yet 🧩
Step 'go to -> (-25, 0, 5)' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
Plan 'Waypoints Trajectory' created —

## Waypoint seletion algorithm

In [6]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,markers,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

# Launch Vehicle

In [7]:
vehicle_path = os.path.expanduser(ardupilot_vehicle_path)

In [8]:
for i in range(n_uavs):
    vehicle_cmd = f"python3 {vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

## Create Plan

In [9]:
uavs=[]
for i,(plan,home) in enumerate(zip(plans,homes)):
    uavs.append(VehicleLogic(sys_id=i+1,
                    home=home,
                    plan= plan))

vehicle 1 created
vehicle 2 created


## Execute Plan

In [10]:
mission_incomplete = n_uavs*[True]
while any(mission_incomplete):
    for i,uav in enumerate(uavs):
        mission_incomplete[i] = not uav.act_plan()

Action 'Pre-Arm Check' is now connected ✅🔗
▶️ Starting Action: Pre-Arm Check
Action 'Pre-Arm Check' is now connected ✅🔗
▶️ Starting Action: Pre-Arm Check
Step 'Check disarmed' is now connected ✅🔗
▶️ Starting Step: Check disarmed
Step 'Check disarmed' is now connected ✅🔗
▶️ Starting Step: Check disarmed
✅ Step: Check disarmed is done
✅ Step: Check disarmed is done
Step 'Check EKF' is now connected ✅🔗
▶️ Starting Step: Check EKF
Step 'Check EKF' is now connected ✅🔗
▶️ Starting Step: Check EKF
EKF is not ready — missing: ATTITUDE, VELOCITY_HORIZ, POS_HORIZ_ABS, POS_VERT_ABS
EKF is not ready — missing: ATTITUDE, VELOCITY_HORIZ, POS_HORIZ_ABS, POS_VERT_ABS
EKF is not ready — missing: ATTITUDE, VELOCITY_HORIZ, POS_HORIZ_ABS, POS_VERT_ABS
EKF is not ready — missing: ATTITUDE, VELOCITY_HORIZ, POS_HORIZ_ABS, POS_VERT_ABS
EKF is not ready — missing: ATTITUDE, VELOCITY_HORIZ, POS_HORIZ_ABS, POS_VERT_ABS
EKF is not ready — missing: ATTITUDE, VELOCITY_HORIZ, POS_HORIZ_ABS, POS_VERT_ABS
EKF is not r

## Kill all related process

In [11]:
# for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
#     os.system(f"pkill -9 -f {process}")